In [1]:
import pandas as pd
import os
os.listdir('/kaggle/input/')

['news-dataset']

In [2]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
    
print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
tf.config.set_visible_devices(physical_devices, 'GPU')

In [4]:
Title=pd.read_excel('/kaggle/input/news-dataset/Title.xlsx')
Text=pd.read_excel('/kaggle/input/news-dataset/Text.xlsx')

In [5]:
# Import required libraries
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

# Define hyperparameters
batch_size = 2
epochs = 100
latent_dim = 256
num_samples = 5
max_encoder_seq_length = 100
max_decoder_seq_length = 30
input_vocab_size = 5000
output_vocab_size = 500

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [6]:

with strategy.scope():
    # Define input and output sequences
    encoder_inputs = Input(shape=(None,))
    decoder_inputs = Input(shape=(None,))
    # Define embedding layer
    embedding_layer = Embedding(input_vocab_size, latent_dim)

    # Define encoder
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(embedding_layer(encoder_inputs))
    encoder_states = [state_h, state_c]

    # Define decoder
    decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder(embedding_layer(decoder_inputs), initial_state=encoder_states)
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    1280000     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 256),        525312      ['embedding[0][0]']          

In [10]:
# Generate training data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize a tokenizer for the text data
text_tokenizer = Tokenizer(num_words=max_encoder_seq_length)
text_tokenizer.fit_on_texts(Text['text'])
text_sequences = text_tokenizer.texts_to_sequences(Text['text'])
encoder_input_data = pad_sequences(text_sequences, maxlen=max_encoder_seq_length)

# Initialize a tokenizer for the title data
title_tokenizer = Tokenizer(num_words=max_decoder_seq_length)
title_tokenizer.fit_on_texts(Title['title'])
title_sequences = title_tokenizer.texts_to_sequences(Title['title'])
decoder_input_data = pad_sequences(title_sequences, maxlen=max_decoder_seq_length, padding='post')

# Prepare the decoder target data by shifting the decoder input by one time step
decoder_target_data = np.zeros((len(title_sequences), max_decoder_seq_length, output_vocab_size), dtype='float32')
for i, sequence in enumerate(title_sequences):
    for t, token in enumerate(sequence[1:]):
        decoder_target_data[i, t, token] = 1.0

In [11]:
# Train model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
17707/17707 [==============================] - 247s 13ms/step - loss: 0.2593 - accuracy: 0.0084 - val_loss: 0.2556 - val_accuracy: 0.0041
Epoch 2/100
17707/17707 [==============================] - 217s 12ms/step - loss: 0.2588 - accuracy: 0.0084 - val_loss: 0.2572 - val_accuracy: 0.0088
Epoch 3/100
17707/17707 [==============================] - 217s 12ms/step - loss: 0.2588 - accuracy: 0.0084 - val_loss: 0.2569 - val_accuracy: 0.0088
Epoch 4/100
17707/17707 [==============================] - 216s 12ms/step - loss: 0.2587 - accuracy: 0.0084 - val_loss: 0.2538 - val_accuracy: 0.0088
Epoch 5/100
17707/17707 [==============================] - 217s 12ms/step - loss: 0.2588 - accuracy: 0.0084 - val_loss: 0.2565 - val_accuracy: 0.0088
Epoch 6/100
17707/17707 [==============================] - 216s 12ms/step - loss: 0.2587 - accuracy: 0.0085 - val_loss: 0.2579 - val_accuracy: 0.0088
Epoch 7/100
 6960/17707 [==========>...................] - ETA: 1:57 - loss: 0.2593 - accuracy: 0.00

KeyboardInterrupt: 

In [ ]:
# Predict titles for new content
encoder_input = np.random.rand(1, max_encoder_seq_length)
decoder_input = np.zeros((1, 1))
decoder_input[0, 0] = output_vocab_size - 1
decoded_title = ''
while decoded_title[-1] != '<EOS>' and len(decoded_title) < max_decoder_seq_length:
    decoder_output = model.predict([encoder_input, decoder_input])
    sampled_token_index = np.argmax(decoder_output[0, -1, :])
    sampled_token = '<EOS>' if sampled_token_index == output_vocab_size - 1 else sampled_token_index
    decoded_title += str(sampled_token)
    decoder_input = np.zeros((1, 1))
    decoder_input[0, 0] = sampled_token

In [ ]:
model.save('finally.hdf5')